In [ ]:
'''Trains a simple convnet on the MNIST dataset.

Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''
from __future__ import print_function

import keras
from keras import backend as K

from keras.models import Sequential
from keras.models import Model

from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Input
from keras.layers import Activation
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Dense

from keras.engine.topology import Layer

from data_manager import ClutteredMNIST
from visualizer import plot_mnist_sample
from visualizer import print_evaluation
from visualizer import plot_mnist_grid
from models import STN

import matplotlib.pyplot as plt
import matplotlib
matplotlib.interactive(True)

from utils import get_initial_weights
from layers import BilinearInterpolation
from data_manager import ClutteredMNIST
from visualizer import plot_mnist_sample
from visualizer import print_evaluation
from visualizer import plot_mnist_grid
from models import STN

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"



In [ ]:
dataset_path = "../datasets/mnist_cluttered_60x60_6distortions.npz"
batch_size = 128
epochs = 12
num_classes = 10

#batch_size = 128
#num_classes = 10
#epochs = 12

# input image dimensions
img_rows, img_cols = 60, 60
input_shape = (img_rows, img_cols, 1)

data_manager = ClutteredMNIST(dataset_path)
train_data, val_data, test_data = data_manager.load()
x_train, y_train = train_data
x_test, y_test = val_data


np.shape(x_train)
np.shape(y_train)
np.shape(y_train)  
np.shape(y_train)  

#y_train = keras.utils.to_categorical(y_train, 10)
np.shape(y_train)
y_train[1,:]

In [ ]:
#Mexer nessa Estrutura
sampling_size=(60,60)

image = Input(shape=input_shape)
locnet = MaxPool2D(pool_size=(2, 2))(image)
locnet = Conv2D(20, (5, 5))(locnet)
locnet = MaxPool2D(pool_size=(2, 2))(locnet)
locnet = Conv2D(20, (5, 5))(locnet)
locnet = Flatten()(locnet)
locnet = Dense(50)(locnet)
locnet = Activation('relu')(locnet)
weights = get_initial_weights(50)
locnet = Dense(6, weights=weights)(locnet)
    
x = BilinearInterpolation(sampling_size)([image, locnet])

x = Conv2D(32, kernel_size=(3, 3), activation='relu')(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(num_classes, activation='softmax')(x)

model = Model(inputs = image, outputs = x)

In [ ]:
model.summary()

In [ ]:
from keras.utils import plot_model
plot_model(model,to_file='CNN_com_STN.png',show_shapes=True)

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


In [ ]:
output_STN = model.get_layer('bilinear_interpolation_1').output
STN_function = K.function([image], [output_STN])

x_test = np.reshape(x_test, (len(x_test), 60, 60, 1))  
np.shape(x_test)

x_test_sample = np.copy(x_test[0:9,:,:,:])
np.shape(x_test_sample)

image_result = STN_function([x_test_sample])
np.shape(image_result)

In [ ]:
a = image_result[0]
np.shape(a)
b = a[0]
np.shape(b)

In [ ]:
#Imagem Teste
plt.figure()
plt.imshow(x_test[2].reshape(60,60))
plt.gray()

#Imagem Transformada
a = image_result[0]
np.shape(a)
b = a[2]
np.shape(b)
plt.figure()
plt.imshow(b.reshape(60,60))
plt.gray()



In [ ]:
plot_mnist_grid(x_test_sample)
plot_mnist_grid(x_test_sample, STN_function)